# Project 1

Max Schrader

AEM 591

#### Resources

https://towardsdatascience.com/on-simulating-non-linear-dynamic-systems-with-python-or-how-to-gain-insights-without-using-ml-353eebf8dcc3

## Install the Required Packages

In [105]:
%pip install pint
%pip install numpy
%pip install plotly
%pip install prettytable

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/linuxbrew/.linuxbrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/linuxbrew/.linuxbrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consider upgrading via the '/home/linuxbrew/.linuxbrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
You should consi

## Pre-Processing

In [215]:
import os
from dataclasses import dataclass
import math
from typing import Union

import pint
import numpy as np
from scipy.integrate import odeint
from prettytable import PrettyTable
# import control

In [7]:
ureg = pint.UnitRegistry()

In [211]:
def pretty_print(array: np.array, headers: list) -> None:
    x = PrettyTable()
    x.field_names = headers
    if len(headers) > 1 and len(array.shape) < 2:
        x.add_row(list(map(lambda x: round(x, 3), array)))
    else:
        for row in array:
#             if len(array.shape) > 1:
            x.add_row(list(map(lambda x: round(x, 3), row)) 
                  if len(array.shape) > 1 else [round(row, 3)])

#             x.add_row(round(row, 3))
    print(x)

### Constants

In [333]:
@dataclass
class Elevator:

    I_m = 700 * ureg.kilogram * ureg.meter ** 2
    K = 100 * ureg.newton * ureg.meter / ureg.ampere
    K_v = 100 * ureg.volt * ureg.second
    L = 0.4 * ureg.H
    m = 500 * ureg.kg
    g = 1 * ureg.gravity
    W = 300 * ureg.kilogram * 1 * ureg.gravity
    r_0 = 3 * ureg.meter
    delta_c = 0.05 * ureg.meter
    R_0 = 5 * ureg.ohm
    delta_r = 5 * ureg.ohm
    t_r = 3 * ureg.second
    
    def r_t(self, x=None, *args, **kwargs) -> pint.unit:
        x = x or self.x
        return (self.delta_c * x / math.pi + self.r_0 ** 2) ** (1/2)  
    
    def R_t(self, t: float) -> pint.unit:
        return self.R_0 + self.delta_r * (1 - math.exp(-t * ureg.second / (self.t_r)))
    

### Given Equations

## Problem 1

Setup two functions to compute the state and outputs in Python using the state-space system
representation above

In [352]:
# g = lambda x:  
class NonLinearElevator(Elevator):
    def __init__(self, x: float, doublet_d_t: float=None, doublet_c: float=None):
        self.doublet_c = doublet_c
        self.doublet_d_t = doublet_d_t
        self.x = x * ureg.meter
    
    def _f2_constant(self, r: float) -> float:
        return 1 / (self.m * r - (self.I_m / r))
    
    def f(self, ) -> object:
        # return inner function because scipy odeint doesn't like OOP 
        def _f(x_t: np.array, t: float) -> np.array:
            x_dot = np.zeros(4)
            
            # Time Variant Fns
            v_t = self.doublet(t)
            
            # X variant FN
            r_t = self.r_t(x_t[0])
            
            x_dot[0] = x_t[1]
            x_dot[1] = (self._f2_constant(r_t) * \
                (
                    (self.W * r_t - self.m * self.g * r_t + self.K * (x_t[2] * ureg.amp)) - 
                        (self.I_m * self.delta_c * (x_t[1] * ureg.mps)**2) / \
                        (2 * math.pi * r_t ** 3)
                )
                       ).to('m/s^2').magnitude
            
            x_dot[2] = \
                (v_t / self.L - \
                ((self.R_t(t)) / self.L * x_t[2] * ureg.ampere) + \
                ((self.K_v * x_t[1] * ureg.mps) / (self.L * r_t))).to('A/s').magnitude
            x_dot[3] = (self.delta_c * x_t[1] * ureg.mps/ (2 * math.pi * r_t)).to('m/s').magnitude
            return x_dot
        return _f
    
    def out(self, ) -> object:
        def _out(x_t: np.array, t: float) -> np.array:
            out_dot = np.zeros(2)
            out_dot[0] = x_t[0]
            out_dot[1] = x_t[1]
            return out_dot
        return _out
    
    def doublet(self, t: float) -> None:
        return {
            0: 0,
            1: self.doublet_c,
            2: -1 * self.doublet_c,
            3: 0,
        }[max(int(t // self.doublet_d_t), 3)] * ureg.volt

## Problem 2

ompute the linearized state-space system about three equilibrium/trim points at 𝑥 = 0,
15, and 30 m for a steady-state value of 𝑅 = 𝑅0 +𝛿𝑅, i.e. 𝑅(𝑡) as 𝑡 → ∞. Analyze the
controllability, observability, and stability in Python for the linearized system.

In [335]:
class LinearElevator(NonLinearElevator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
#         self.x = x * ureg.meter
        
    def R_t(self, *args, **kwargs) -> pint.unit:
        return 10 * ureg.ohm
    
    """ Doing the linearized points as @property is not computationaly 
        efficient but makes the code most readable for me """
    @property
    def x_bar(self, ) -> float:
        return 0
    
    @property
    def v_bar(self, ) -> float:
        return ((self.m * self.g - self.W) * (self.R_0 + self.delta_r)) / self.K * self.r_t() 
    
    @property
    def i_bar(self, ) -> float:
        return self.v_bar / (self.R_0 + self.delta_r)
    
    @property
    def r_bar(self, ) -> float:
        return self.K / (self.m * self.g - self.W) * self.i_bar
    
    def df1_dx(self, *args, **kwargs) -> np.array:
        return np.array(
        [
            0, 
            1, 
            0, 
            0
        ])
    
    def df2_dx(self, ) -> np.array:
        return np.array(
            [
                0, 
                0, # (self.I_m * self.delta_c * x_t[1]) / (math.pi * self.m * r_t **4 + x_t[2] * math.pi * r_t ** 2) commented out because x_bar = 0
                (self.K / (self.m * self.r_bar - (self.I_m / self.r_bar))).magnitude,
                ((2 * self.I_m * math.pi * self.r_bar ** 4 * (self.g * self.m - self.W)) / (math.pi * self.r_bar ** 3 * (self.m * self.r_bar**2 - self.I_m) ** 2)).magnitude
            ]
        )
    
    def df3_dx(self, ) -> np.array:
        return np.array(
            [
                0,
                ((-1 * self.K) / (self.L * self.r_bar)).magnitude,
                (self.R_t() / self.L).magnitude,
                0 # because x_bar = 0
            ]
        )
    
    def df4_dx(self, ) -> np.array:
        return np.array(
            [
                0,
                (self.delta_c / (2 * math.pi * self.r_bar)).magnitude,
                0,
                0
            ]
        )
    
    @property
    def A(self, ) -> np.array:
        return np.array(
            [
                self.df1_dx(),
                self.df2_dx(),
                self.df3_dx(),
                self.df4_dx()
            ]
        )
    
    @property
    def B(self, ) -> np.array:
        return np.array(
            [
                0,
                0,
                (1 / self.L).magnitude,
                0
            ]
        )
    
    @property
    def C(self, ) -> np.array:
        return np.array(
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        )
        
    def D(self, ) -> np.array:
        return np.array(
            [0, 0, 0, 0]
        )

### Checking the Units

In [190]:
elevator = LinearElevator(x=100)
# this should never be done
self = elevator

#### df2_dx

In [191]:
r_const = ((2 * self.I_m * math.pi * self.r_bar ** 4 * (self.g * self.m - self.W)) / (math.pi * self.r_bar ** 3 * (self.m * self.r_bar**2 - self.I_m) ** 2))
i_const = self.K / (self.m * self.r_bar - (self.I_m / self.r_bar))

In [192]:
final = ((i_const * 1 * ureg.ampere) + (r_const * 1 * ureg.meter))
assert final.units == 1 * ureg.mps / ureg.second

#### df3_dx

In [194]:
x_dot_const = (-1 * self.K) / (self.L * self.r_bar)
i_const = self.R_t() / self.L

In [195]:
final = (x_dot_const * 1 * ureg.mps) + (i_const * 1 * ureg.ampere)
assert final.units == 1 * ureg.amp / ureg.second

#### df4_dx

In [196]:
x_dot_const = self.delta_c / (2 * math.pi * self.r_bar)

In [197]:
final = x_dot_const * 1 * ureg.mps
assert final.units == 1 * ureg.mps

### State Space at $x = 0$

In [198]:
elevator = LinearElevator(x=0)
pretty_print(elevator.A, ['x', 'x_dot', 'i', 'r'])

+-----+---------+-------+-------+
|  x  |  x_dot  |   i   |   r   |
+-----+---------+-------+-------+
| 0.0 |   1.0   |  0.0  |  0.0  |
| 0.0 |   0.0   | 0.079 | 0.058 |
| 0.0 | -83.333 |  25.0 |  0.0  |
| 0.0 |  0.003  |  0.0  |  0.0  |
+-----+---------+-------+-------+


In [199]:
pretty_print(elevator.B, ['v'])

+-----+
|  v  |
+-----+
| 0.0 |
| 0.0 |
| 2.5 |
| 0.0 |
+-----+


In [200]:
elevator.v_bar.to('V')

588.399 <Unit('volt')>

#### Finding the stability

In [201]:
np.linalg.eigvals(elevator.A)

array([ 0.00000000e+00,  2.47340121e+01,  2.66573018e-01, -5.85073228e-04])

In [202]:
np.linalg.matrix_rank(elevator.A)

3

## Problem 3

Simulate the nonlinear system with doublet commands on $\dot{v}$, both up and down, for initial conditions close and far from the linearized states, but including $t$. Use input values which keep the elevator velocity reasonable

In [336]:
x = 0
doublet_magnitude = LinearElevator(x).v_bar
doublet_dt = 10

t = np.linspace(0, 20, 1000)
elevator = NonLinearElevator(x, doublet_c=doublet_magnitude, doublet_d_t=doublet_dt)

# compose the initial conditions
x0 = np.array([elevator.x.magnitude, 0, 0, elevator.r_t(elevator.x).magnitude])
pretty_print(x0, headers=['x', 'x_dot', 'i', 'r'])

+-----+-------+-----+-----+
|  x  | x_dot |  i  |  r  |
+-----+-------+-----+-----+
| 0.0 |  0.0  | 0.0 | 3.0 |
+-----+-------+-----+-----+


In [337]:
# s = odeint(elevator.f(), y0=x0, t=t)
self = elevator

x_t = np.array([1, 1, 1, 1])
t = 0

In [347]:
(self.doublet(t) / self.L).dimensionality

<UnitsContainer({'[current]': 1, '[time]': -1})>

In [351]:
self.delta_c * x_t[1]*ureg.mps / (2 * math.pi * r_t)

0.0026525823848649226 <Unit('meter_per_second')>

In [326]:
self.K.to('Vs')

UndefinedUnitError: 'Vs' is not defined in the unit registry

In [302]:
self.doublet(t) / self.L - \
                ((self.R_t(t)) / self.L * x_t[2] * ureg.ampere) - \
                (self.K * x_t[1] * ureg.mps) / (self.L * r_t)

DimensionalityError: Cannot convert from 'second / henry' ([current] ** 2 * [time] ** 3 / [length] ** 2 / [mass]) to 'ampere * ohm / henry' ([current] / [time])

In [282]:
self.I_m = 700 * ureg.kg * ureg.meter **2

In [283]:
self.delta_c

0.05 <Unit('meter')>

In [286]:
(self.W * r_t - self.m * self.g * r_t + self.K * 1 * ureg.amp) + ((self.I_m * self.delta_c * (1 * ureg.mps)**2) / (2 * math.pi * r_t ** 3))

-589.7817999048333 <Unit('kilogram * meter * standard_gravity')>

<UnitsContainer({'[length]': 2, '[mass]': 1, '[time]': -2})>

## Problem 4

Simulate the linearized system with doublet commands on $dot{v}$, both up and down, for initial conditions close to the linearized states. Use input values which keep the elevator velocity reasonable

## Problem 5

Write a few paragraphs summarizing the results of these simulations and any analysis. Make sure to compare the linear simulations to the nonlinear.